**url**

https://www.zyduslife.com/news#pressrelease

In [1]:
import pandas as pd,requests,bs4,re,time,io,pdftotext
from selenium.webdriver.common.by import By
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
SITE_NAME='Zyduslife'

DOMAIN = "https://www.zyduslife.com"

SITE_LINK="https://www.zyduslife.com/news#pressrelease"

In [3]:
def remove_esc_chars(text):
    return text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

In [4]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [5]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [6]:
#addding timeout so that it won't force the work and scrape everything
timeout = 100

In [7]:
url = SITE_LINK
#getting the url
driver.get(url)

#letting driver wait for a time
driver.implicitly_wait(timeout)

#accepting cookies
driver.find_element_by_xpath('''//*[@id="qq"]/div/div/div/div/button''').click()
driver.implicitly_wait(timeout)

#clicking on all
all_options = driver.find_elements(By.XPATH, '//option[@value="0"]')
for i in all_options:
    i.click()

time.sleep(5)

#get the titles ,urls and the dates
elements = driver.find_elements(By.CLASS_NAME, "col-sm-4")

In [8]:
article_list = []

total_elements = len(elements[::-1][:20])
for x in elements[::-1][:20]:
    y = x.find_elements(By.TAG_NAME, 'a')
    for z in y:
        #title,published_date,link,author,thumbnail
        title = z.find_element(By.TAG_NAME, 'p').text
        published_date = z.find_element(By.TAG_NAME, 'span').text
        link = z.get_attribute('href')
        author = SITE_NAME
        thumbnail = "https://www.zyduslife.com/public/images/zydus-logo.png"
        
        #text
        r = requests.get(link)
        f = io.BytesIO(r.content)
        reader = pdftotext.PDF(f)
        text = ""
        for x in range(0,len(reader)):
            text += remove_esc_chars(reader[x])
            
        article = (published_date.strip(),title.strip(),text.strip(),link.strip(),thumbnail.strip(),author.strip())
        article_list.append(article)
        print(published_date,title)
        total_elements -= 1
        print(f"{total_elements} pages remaining..",end ="\r")

22 December 2021 Zydus Cadila receives tentative approval from USFDA for Pimavanserin Tablets
16 December 2021 Zydus receives Orphan Drug Designation from US FDA for ZY-19489, a novel compound to treat malaria
13 December 2021 Zydus to begin Phase II (a) clinical trial of ZYIL1, a novel oral NLRP3 inflammasome inhibitor in patients with CryopyrinAssociated Periodic Syndrome (CAPS) in Australia
10 December 2021 Zydus Cadila receives tentative approval from USFDA for Cariprazine Capsules
10 December 2021 Zydus announces EPICS-IIITM Phase 2(b)/3 adaptive pivotal clinical trial of Saroglitazar Mg in PBC
06 December 2021 Together We Fly Quilt Press Note
23 November 2021 Zydus develops new treatment for patients suffering from Chronic Kidney Disease (CKD), submits first New Drug Application (NDA) to DCGI
18 November 2021 Zydus Cadila gets 180-day exclusivity for Nelarabine injection, announces final approval from USFDA
10 November 2021 Zydus Cadila receives final approval from USFDA for Gly

In [9]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df.head()

,date,title,article,url,thumbnail,author
0,22 December 2021,Zydus Cadila receives tentative approval from ...,Zydus Cadila receives tentative approval from ...,https://www.zyduslife.com/public/pdf/pressrele...,https://www.zyduslife.com/public/images/zydus-...,Zyduslife
1,16 December 2021,Zydus receives Orphan Drug Designation from US...,Zydus receives Orphan Drug Designation from US...,https://www.zyduslife.com/public/pdf/pressrele...,https://www.zyduslife.com/public/images/zydus-...,Zyduslife
2,13 December 2021,Zydus to begin Phase II (a) clinical trial of ...,Zydus to begin Phase II (a) clinical trial of ...,https://www.zyduslife.com/public/pdf/pressrele...,https://www.zyduslife.com/public/images/zydus-...,Zyduslife
3,10 December 2021,Zydus Cadila receives tentative approval from ...,Zydus Cadila receives tentative approval from ...,https://www.zyduslife.com/public/pdf/pressrele...,https://www.zyduslife.com/public/images/zydus-...,Zyduslife
4,10 December 2021,Zydus announces EPICS-IIITM Phase 2(b)/3 adapt...,Zydus announces EPICS-IIITM Phase 2(b)/3 adapt...,https://www.zyduslife.com/public/pdf/pressrele...,https://www.zyduslife.com/public/images/zydus-...,Zyduslife


In [10]:
#to csv
temp_df.to_csv(f'{SITE_NAME} news.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} news.json')